In [16]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime
import random
import time
import cloudscraper
import warnings


warnings.filterwarnings("ignore")

scraper = cloudscraper.CloudScraper()

agents = open('desktop_user_agent.txt').readlines()
user_agents_list = [line[:-2] for line in agents]

URL = 'http://kremlin.ru'

n = 1
df = pd.DataFrame(columns=['doc_type','name','number','date','description','text'])

In [3]:
def date_is_too_old(date_start, date):
    return date < date_start

def date_is_too_early(date_finish, date):
    return date > date_finish

def make_date(date):
    d = date.split("-")
    d = datetime.datetime(int(d[0]), int(d[1]), int(d[2]))
    return d

In [4]:
date_start = make_date(input("type start date in yyy-mm-dd format"))

date_finish = make_date(input("type finish date in yyy-mm-dd format"))

type start date in yyy-mm-dd format 2022-09-01
type finish date in yyy-mm-dd format 2022-09-29


In [5]:
if date_start < date_finish:
    pass
else:
    print('dates aren\'t correct')

In [17]:
while(1):
    
    time.sleep(2)
    
    page_url = URL + '/acts/bank/page/'+str(n)
    
    response = scraper.get(page_url, headers={'User-Agent': random.choice(user_agents_list)})
    
    print('page '+str(n)+' response = ', response)
    
    text = bs(response.text,"lxml")
    
    page_dates = [make_date(str(d).split("\"")[1]) for d in text.find_all('time')]
    
    if False not in [date_is_too_old(date_start, d) for d in page_dates]:
        print('that\'s all, break')
        break
        
    if False not in [date_is_too_early(date_finish, d) for d in page_dates]:
        print('date is too early, continue')
        n += 1
        continue
        
    events = text.find_all('h3', 'hentry__title hentry__title_special')
    
    for event in events:
        time.sleep(2)
        date = make_date(str(event.find('time')).split("\"")[1])
        if date_is_too_early(date_finish, date):
            continue
        if date_is_too_old(date_start, date):
            break
            
        description = event.find('span', 'hentry__meta_acts')
        if not description:
            description = bs("")
        description = description.text
        
        text_url = URL + str(event.find('a')).split('"')[1]
        
        start_name = str(event.find('a')).find('>')+1
        end_name = str(event.find('a')).find('<s')-1
        start_number = str(event.find('a')).find('№')+2
        end_doc_type = str(event.find('a')).find('от ')-1
        
        name = str(event.find('a'))[start_name:end_name]
        
        number = str(event.find('a'))[start_number:end_name]
        
        doc_type = str(event.find('a'))[start_name:end_doc_type]
        
        response = scraper.get(text_url, headers={'User-Agent': random.choice(user_agents_list)})
        
        print('response = ', response)
        
        doc_text = bs(response.text,"lxml").find('div', 'reader_act_body').text
        
        df = df.append({'doc_type':doc_type,'name':name,'number':number,'date':date,'description':description,'text':doc_text}, ignore_index=True)
    n += 1
df.to_csv("table.csv", index=False)

page 1 response =  <Response [200]>
date is too early, continue
page 2 response =  <Response [200]>
date is too early, continue
page 3 response =  <Response [200]>
date is too early, continue
page 4 response =  <Response [200]>
date is too early, continue
page 5 response =  <Response [200]>
date is too early, continue
page 6 response =  <Response [200]>
date is too early, continue
page 7 response =  <Response [200]>
date is too early, continue
page 8 response =  <Response [200]>
date is too early, continue
page 9 response =  <Response [200]>
date is too early, continue
page 10 response =  <Response [200]>
date is too early, continue
page 11 response =  <Response [200]>
date is too early, continue
page 12 response =  <Response [200]>
date is too early, continue
page 13 response =  <Response [200]>
date is too early, continue
page 14 response =  <Response [200]>
response =  <Response [200]>
response =  <Response [200]>
response =  <Response [200]>
response =  <Response [200]>
response = 